In [44]:
import pandas as pd
import numpy as np
import json
import gzip
import seaborn as sns
from pandas.io.json import json_normalize

## Open Catalog dataset

In [2]:
catalog = pd.read_csv('data/catalog.gz')

In [3]:
catalog.head()

,pid,current_price,original_price,category,sub_category,sub_sub_category
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,NaN,c1bd5fd999bd577743936898ada96496b547af3c,f08770a96fb546673053ab799f5ea9cada06c06a,2d2c44a2d8f18a6271f0e8057313af68a46d0f24
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,NaN,37b2661cc181c987f68770e43990152026920ba5,11e623a37e87cf7995c466723ec99688d55cae8c,11e623a37e87cf7995c466723ec99688d55cae8c
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,c37df0ae71b97699a478def3001a3516a905a51d,f08770a96fb546673053ab799f5ea9cada06c06a,d6605426aaa703bf19874c0caac79a661b73de33
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,c1bd5fd999bd577743936898ada96496b547af3c,6d9d48ae11ee1909235f31e9bfe5d36aa1462cb3,9a2e3cb56f6a1756fd35a4fd70172a67ecf13639
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,2541a58f702844477aab540e5df7b859a1e3d5de


## Open dataset with a JSON object per line with NaN

In [4]:
%%time
with gzip.open('data/test.gz', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
data = pd.DataFrame(json_records)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 58.3 ms


In [5]:
data.head()

,date,event_type,page_type,productId,products,source,timestamp,uid
0,NaN,pageview,cart,NaN,NaN,NaN,2016-01-01,54cfe1703c536e98bf02c7c4cd5b4e280f07d74c
1,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,c808d5ecb73bee00a9730064b7652bec10778f20
2,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,7ba0cbaaf2000d440b474ff9182383c5525d0d56
3,NaN,pageview,confirmation,NaN,NaN,NaN,2016-01-02,3b2790a1f746e59298c8c277e3ec2279343ce52d
4,NaN,pageview,checkout,NaN,NaN,NaN,2016-01-02,3b2790a1f746e59298c8c277e3ec2279343ce52d


## Open only purchase events from data dataset

In [6]:
%%time
with open('data/purchase_data', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
purchase = pd.DataFrame(json_records)

CPU times: user 652 ms, sys: 36 ms, total: 688 ms
Wall time: 747 ms


In [7]:
purchase.head()

,date,event_type,gender,products,source,uid
0,2017-03-27 14:19:00,purchase,M,[{u'pid': u'db3eae1855619bd4462848bb7b473b2193...,desktop,1f9acd4f729c04e18e5d72bebca5e5524e67687e
1,2016-05-26 11:21:40,purchase,M,[{u'pid': u'418e19155782fc7f12ced8332c8ed025ae...,NaN,07be0ebd1e6f2412e4a9a3f679ded05796a1e225
2,2016-03-05 06:52:27,purchase,M,[{u'pid': u'4b17c3aa28d53208c9fd147057b40fdd81...,NaN,3cf9949b8f90daadd8f67ec46214e64d6b9cfa5b
3,2016-11-07 11:36:00,purchase,M,[{u'pid': u'acd9858f0d4d4c61ee783080a1a305b4f0...,desktop,7f1c45f10de66a7023bfa9f9ed053c5b3ae03f56
4,2016-08-28 20:48:58,purchase,M,[{u'pid': u'fca1dc5059edb8a8ee2fa424ce7a01958b...,NaN,79ddd2b0c65947a8a11249c2fa71499055507e11


## Unnest products

In [42]:
%%time
with open('data/purchase_data', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        products = record.pop('products')
        record.pop('source')
        record.pop('event_type')
        for obj in products:
            new_record = record.copy()
            new_record.update(obj)
            json_records.append(new_record)
purchase = pd.DataFrame(json_records)

CPU times: user 596 ms, sys: 0 ns, total: 596 ms
Wall time: 594 ms


In [43]:
purchase.head()

,date,gender,pid,quantity,uid
0,2017-03-27 14:19:00,M,db3eae1855619bd4462848bb7b473b219345ee87,1.0,1f9acd4f729c04e18e5d72bebca5e5524e67687e
1,2017-03-27 14:19:00,M,3aed9b0313f9226111de8aeabaedccf8db07d428,1.0,1f9acd4f729c04e18e5d72bebca5e5524e67687e
2,2016-05-26 11:21:40,M,418e19155782fc7f12ced8332c8ed025aec227a5,1.0,07be0ebd1e6f2412e4a9a3f679ded05796a1e225
3,2016-03-05 06:52:27,M,4b17c3aa28d53208c9fd147057b40fdd8108f790,1.0,3cf9949b8f90daadd8f67ec46214e64d6b9cfa5b
4,2016-03-05 06:52:27,M,31771f432cbc8e7ee3140f70b1e866fb5a2d739a,1.0,3cf9949b8f90daadd8f67ec46214e64d6b9cfa5b


### Using json_normalize

In [48]:
%%time
with open('data/purchase_data', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
purchase = json_normalize(json_records, 'products', ['uid','date','gender'])

CPU times: user 760 ms, sys: 12 ms, total: 772 ms
Wall time: 770 ms


In [49]:
purchase.head()

,pid,quantity,date,gender,uid
0,db3eae1855619bd4462848bb7b473b219345ee87,1.0,2017-03-27 14:19:00,M,1f9acd4f729c04e18e5d72bebca5e5524e67687e
1,3aed9b0313f9226111de8aeabaedccf8db07d428,1.0,2017-03-27 14:19:00,M,1f9acd4f729c04e18e5d72bebca5e5524e67687e
2,418e19155782fc7f12ced8332c8ed025aec227a5,1.0,2016-05-26 11:21:40,M,07be0ebd1e6f2412e4a9a3f679ded05796a1e225
3,4b17c3aa28d53208c9fd147057b40fdd8108f790,1.0,2016-03-05 06:52:27,M,3cf9949b8f90daadd8f67ec46214e64d6b9cfa5b
4,31771f432cbc8e7ee3140f70b1e866fb5a2d739a,1.0,2016-03-05 06:52:27,M,3cf9949b8f90daadd8f67ec46214e64d6b9cfa5b
